In [3]:
# !pip install --upgrade plotly plotnine
# !pip install fastinference

In [4]:
import os
import pickle
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from fastai.tabular.all import * 
# from fastinference.tabular import *
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder, StandardScaler, SplineTransformer


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts

warnings.simplefilter("ignore")

In [5]:
def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True)

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
#     os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)
seed_everything(seed=42)

In [6]:
set_seed(42)
seed_everything(seed=42)

df_train = pd.read_csv("/kaggle/input/ecm-itu-zindi-kp-data/imgs_2023071012133740345.csv",parse_dates=['Time'])
df_test = pd.read_csv("/kaggle/input/ecm-itu-zindi-kp-data/imgs_202307101549519358.csv",parse_dates=['Time'])
df_cell = pd.read_csv("/kaggle/input/ecm-itu-zindi-kp-data/imgs_2023071012130978799.csv",parse_dates=['Time'])
df_bs = pd.read_csv("/kaggle/input/ecm-itu-zindi-kp-data/imgs_2023071012123392536.csv")
df_features = df_cell.merge(df_bs,on=['BS','CellName'],how='outer')
df_features = df_features[df_features['CellName']=='Cell0'].reset_index(drop=True)
df_test['split'] = 'test'
df_train['split'] = 'train'
df_total = pd.concat([df_train,df_test],ignore_index=True)
df_total = df_total.merge(df_features,on=['BS','Time'],how='left')
df_total['ID'] = df_total['Time'].astype(str)+"_"+df_total['BS']
df_total['BS'] = df_total['BS'].str.replace(r'[a-zA-Z_]', '', regex=True).astype(int)
for col in ['RUType','Mode']:
    df_total[col] = df_total[col].str.replace(r'[a-zA-Z]', '', regex=True).astype(int)

df_total.sort_values(['BS','Time'], ascending=True,ignore_index=True,inplace=True)
df_total['day'] = df_total['Time'].dt.day
df_total['weekday_number'] = df_total['Time'].dt.weekday
df_total['hour'] = df_total['Time'].dt.hour

hour_df = df_total[['hour']].copy()
splines = periodic_spline_transformer(24, n_splines=12).fit_transform(hour_df)
splines_df = pd.DataFrame(splines,columns=[f"hour_spline_{i}" for i in range(splines.shape[1])])
df_total = pd.concat([df_total,splines_df],axis=1)

df_total = df_total.sort_values(['BS','Time'],ascending=True,ignore_index=True)
all_shits = list(np.arange(1,4)) # 
for shift_i in tqdm(all_shits):
    for col in ['load','ESMode1','ESMode2','ESMode3','ESMode6','Time','Energy']:
        df_total[f'{col}_T-{shift_i}'] = df_total.groupby(['BS'])[col].shift(shift_i)        
for shift_i in tqdm(all_shits):
    df_total[f'Time_T-{shift_i}_hours_elapsed'] = (df_total[f'Time_T-{shift_i}']-df_total['Time']).dt.total_seconds() / 3600
    del df_total[f'Time_T-{shift_i}']
print(df_total.shape)

num_bins = 100
df_total['load_bin'] = pd.cut(df_total['load'],bins=[round(i,2) for i in list(np.arange(0,1.01,0.01))],labels=[f'{i}' for i in range(num_bins)])
df_total['load_bin'] = df_total['load_bin'].astype(float).fillna(-1).astype(int)

print(df_total.shape)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/ecm-itu-zindi-kp-data/imgs_2023071012133740345.csv'

In [ ]:
sns.displot(df_total['Energy'])

In [ ]:
df_total.groupby(['Time'])[['Energy']].mean().plot()

In [ ]:
df_total.groupby(['load'])[["Energy"]].mean().plot()

In [ ]:
set_seed(42)
seed_everything(seed=42)

from scipy.signal import savgol_filter as sg
from scipy.signal import sosfiltfilt, butter, sosfilt, sosfilt_zi

def add_sg(df):
    w = 5 #5
    p = 3 #2
    for si in tqdm(df.BS.unique()):
        index = df.BS == si
        df.loc[index, 'load_smooth'] = sg(df[index].load, w, p)
        df.loc[index, 'load_diff'] = sg(df[index].load, w, p, 1)
        df.loc[index, 'load_diff2'] = sg(df[index].load, w, p, 2)
        df.loc[index, 'load_diff3'] = sg(df[index].load, w, p, 3)

add_sg(df_total)
print(df_total.shape)

def add_sosfiltfilt(df):
    for si in tqdm(df.BS.unique()):
        index = df.BS == si
        sos=butter(4, 0.125, output='sos')
        sos8 = butter(8, 0.125, output='sos')
        zi = np.array(df[index].load[:4]).mean() * sosfilt_zi(sos8)
        df.loc[index, 'load_sosfiltfilt'] = sosfiltfilt(sos,df[index].load)
        df.loc[index, 'load_sosfilt'], _ = sosfilt(sos8, df[index].load, zi=zi)

add_sosfiltfilt(df_total)
print(df_total.shape)

In [ ]:
set_seed(42)
seed_everything(seed=42)

df_total.drop(columns=['w','CellName','ESMode4','Time'],inplace=True)

id_variable = 'ID'
version_nb = 'v4'
TARGET = 'Energy'

train_df = df_total[df_total['split']=='train'].reset_index(drop=True)
test_df = df_total[df_total['split']=='test'].reset_index(drop=True)
train_cols = [i for i in train_df if i not in ['Time','CellName','ID','Energy','split','w','BS','ESMode6']]

categorical_cols = ['RUType','Mode','load_bin']

print(train_df[train_cols].shape, test_df[train_cols].shape)

In [ ]:
set_seed(42)
seed_everything(seed=42)

remove_non_unique_cols = []
print('train single values ..')
for col in train_df:
    if col in train_cols and col not in ['ESMode5']:
        if train_df[col].nunique()<=1:
            remove_non_unique_cols.append(col)
            print(col,":",train_df[col].nunique())
print('test single values ..')
for col in test_df:
    if col in train_cols and col not in ['ESMode5']:
        if test_df[col].nunique()<=1:
            remove_non_unique_cols.append(col)
            print(col,":",test_df[col].nunique())

print('\n',remove_non_unique_cols)

for col in train_df:
    if col in train_cols:
        if train_df[col].isnull().sum()/len(train_df)>=0.95:
            print(col,":",train_df[col].isnull().sum()/len(train_df))
            remove_non_unique_cols.append(col)

for col in test_df:
    if col in train_cols:
        if test_df[col].isnull().sum()/len(test_df)>=0.95:
            print(col,":",test_df[col].isnull().sum()/len(test_df))
            remove_non_unique_cols.append(col)
            
print(len(train_cols))
train_cols = [col for col in train_cols if col not in remove_non_unique_cols]
print(len(train_cols))
print(train_cols)


In [ ]:
set_seed(42)
seed_everything(seed=42)

Nfold = 10
Inference=False

if Inference:
    print('Data has been splitted...')
else:
    train_df['fold'] = 0
    strafy_bin = train_df['BS'].astype('int')

    skf = GroupKFold(n_splits = Nfold)
    for i, (_, train_index) in enumerate(skf.split(train_df.index,train_df.index, strafy_bin)):
        train_df.loc[train_index, 'fold'] = i  

### FastAI model

In [ ]:
set_seed(42)
seed_everything(seed=42)

def mae(preds, targs):
    x = (targs-preds)
    return (abs(x)).mean()

def mape(preds, targs):
    x = (targs-preds)/targs
    return (abs(x)).mean()

def fit_fastai(Nfolds, train_df, test_df, train_cols, cat_feats, TARGET, model_path):
    
    oof_pred_fastai = np.zeros(train_df.shape[0], dtype=np.float32)
    pred_fastai = np.zeros(test_df.shape[0], dtype=np.float32)
    scores = []
    scores_pvt = []
    
    train_df_fast = train_df[train_cols + [TARGET]].copy()
    test_df = test_df.copy()
    for col in train_df_fast.columns:
        if col not in cat_feats:
            train_df_fast[col] = train_df_fast[col].fillna(0)
            test_df[col] = test_df[col].fillna(0)
        else:
            # Impute missing categorical values with the most frequent category
            most_frequent_category = train_df_fast[col].mode().iloc[0]
            train_df_fast[col] = train_df_fast[col].fillna(most_frequent_category)
            test_df[col] = test_df[col].fillna(most_frequent_category)

    train_df_fast[cat_feats] = train_df_fast[cat_feats].astype('category')
    test_df[cat_feats] = test_df[cat_feats].astype('category')
    
    cont_nn = train_cols.copy()
    for col in cat_feats:
        cont_nn.remove(col)

    cat_nn = cat_feats

    layers =  [256, 512, 1024, 512, 256] #[256, 512, 1024, 512, 256]

    val_pct, tst_preds = L(), L()

    for fold in range(Nfolds):
        print("*"*10, f'Fold-{fold+1}', "*"*10)
        train_idx = train_df.loc[train_df['fold']!=fold, :].index
        valid_idx = train_df.loc[train_df['fold']==fold, :].index
        splits = (L(list(train_idx)), L(list(valid_idx)))
        dls = TabularPandas(train_df_fast, [Categorify, Normalize], cat_nn, cont_nn, splits = splits, y_names=TARGET,reduce_memory=False).dataloaders(1024)
        learn = tabular_learner(dls, layers=layers, n_out=1, y_range = (0,100),loss_func = mae, metrics=AccumMetric(mae))
#         print(learn.summary())
#         learn.lr_find(suggest_funcs=(slide, valley))
#         if os.path.isfile(model_path + f'models/nn_model_{fold}.pth'):
#             learn = tabular_learner(dls, layers=layers, n_out=1, path = model_path)
#             learn.load(f'nn_model_{fold}')
#         else:
        learn.fit_one_cycle(100, 2e-3, cbs=SaveModelCallback(monitor='mae', comp=np.less, fname=f'nn_model_{fold}'))

        val_df = train_df.loc[train_df['fold']==fold]
        val_dl = dls.test_dl(val_df[train_cols].fillna(0))
        
        preds, _ = learn.get_preds(dl=val_dl)
        oof_pred_fastai[val_df.index] = preds.squeeze().numpy()
       
        score = mean_absolute_error(val_df[TARGET], oof_pred_fastai[val_df.index])
        score_pvt = mean_absolute_percentage_error(val_df[TARGET], oof_pred_fastai[val_df.index])
        
        scores.append(score)
        scores_pvt.append(score_pvt)
        print(f'MAE for Fold-{fold+1}:', np.round(score, 3))
        print(f'MAPE for Fold-{fold+1}:', np.round(score_pvt, 3))
        
        test_dl = dls.test_dl(test_df[train_cols])
        preds, _ = learn.get_preds(dl=test_dl)
        pred_fastai += preds.squeeze().numpy()/Nfolds

#         display(test_df[train_cols].iloc[-5:])
        
#         exp = ShapInterpretation(learn, test_df[train_cols].iloc[-5:])
#         exp.summary_plot()


    score = mean_absolute_error(train_df[TARGET],oof_pred_fastai)
    score_pvt = mean_absolute_percentage_error(train_df[TARGET],oof_pred_fastai)
    
    print(f'OOF MAE:', np.round(score, 3))
    print(f'Average MAE:', f'{np.round(np.mean(scores), 3)}+/-{np.round(np.std(scores), 3)}')
    
    print(f'OOF MAPE:', np.round(score_pvt, 3))
    print(f'Average MAPE:', f'{np.round(np.mean(scores_pvt), 3)}+/-{np.round(np.std(scores_pvt), 3)}')

    return oof_pred_fastai, pred_fastai

In [ ]:
set_seed(42)
seed_everything(seed=42)

if Inference:
    model_path = '../input/ecm-output-of-final-notebook/'
else:
    model_path = './'

oof_pred_fastai, pred_fastai = fit_fastai(Nfold, train_df, test_df, train_cols, categorical_cols, TARGET, model_path)

train_df = train_df.copy()
train_df['oof_fastai'] = oof_pred_fastai

In [ ]:
set_seed(42)
seed_everything(seed=42)

submission_fastai = pd.DataFrame(data = {'Time': test_df['ID'].values, 'Energy': pred_fastai})
print(submission_fastai.head())
submission_fastai.to_csv(f'submission_fastai.csv', index=False)

In [ ]:
# OOF MAE: 0.686
# Average MAE: 0.686+/-0.048
# OOF MAPE: 0.027
# Average MAPE: 0.027+/-0.003

### Keras Model

In [ ]:
from tensorflow import keras
from keras import backend as K
import tensorflow as tf

from tensorflow import keras
from keras import backend as K
from keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from keras.layers import Activation
from keras.backend import sigmoid
from keras import backend as K

set_seed(42)
seed_everything(seed=42)
tf.random.set_seed(42)


In [ ]:
set_seed(42)
seed_everything(seed=42)
tf.random.set_seed(42)

def mean_absolute_error(y_true, y_pred):
         return K.mean(K.abs( (y_true - y_pred)))

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=50, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.25, patience=8, verbose=0,
    mode='min')

def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

def mish(x, beta = 1):
    return (x * K.tanh(K.softplus(x)))


get_custom_objects().update({'swish': Activation(swish)})
get_custom_objects().update({'mish': Activation(mish)})

In [ ]:
set_seed(42)
seed_everything(seed=42)
tf.random.set_seed(42)

def base_model(hidden_units, embedding_size, train_cols, 
               categorical_cols, uniques):

    n_cont=0
    initial_inputs=[]
    for col in categorical_cols:
        temp_input = keras.Input(shape=(1,), name=col)
        n_cont+=1
        initial_inputs.append(temp_input)
        
    num_input = keras.Input(shape=(len(train_cols)-n_cont,), name='num_data')
    initial_inputs.append(num_input)

    #embedding, flatenning and concatenating
    all_inputs=[]
    for i, col in enumerate(categorical_cols):
        temp_embedded = keras.layers.Embedding(int(uniques[col]), embedding_size, 
                                               input_length=1, name=f'{col}_embedding')(initial_inputs[i])
        temp_flattened = keras.layers.Flatten()(temp_embedded)
        all_inputs.append(temp_flattened)
    
    all_inputs.append(num_input)
    out = keras.layers.Concatenate()(all_inputs)
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:
        out = keras.layers.Dense(n_hidden, activation='swish')(out)

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)

    model = keras.Model(inputs = initial_inputs, outputs = out)
    
    return model

In [ ]:
set_seed(42)
seed_everything(seed=42)
tf.random.set_seed(42)

def fit_keras_nn(Nfolds, train_df, test_df, train_cols, cat_cols, TARGET, 
                   model_path):
    
    model_name = 'Keras_NN'
    oof_keras = np.zeros(train_df.shape[0])
    pred_keras = np.zeros(test_df.shape[0])
    scores=[]
    scores_pvt = []
    
    train_df = train_df.copy()
    test_df = test_df.copy()
    train_df = train_df.fillna(0)
    test_df = test_df.fillna(0)
    
    cont_cols = train_cols.copy()
    uniques={}
    for col in cat_cols:
        cont_cols.remove(col)
        le = LabelEncoder()
        le.fit(pd.concat([train_df[col], test_df[col]]))
        train_df[col] = le.transform(train_df[col].values)
        test_df[col] = le.transform(test_df[col].values)
        uniques[col] = len(pd.concat([train_df[col], test_df[col]], axis=0).unique())
        
    scaler = StandardScaler().fit(pd.concat([train_df[cont_cols], test_df[cont_cols]], axis=0))       
    train_df[cont_cols] = scaler.transform(train_df[cont_cols].values)
    test_df[cont_cols] = scaler.transform(test_df[cont_cols].values)
    
    test_inputs=[]
    for col in cat_cols:
        test_inputs.append(test_df[col].values)
        
    test_inputs.append(test_df[cont_cols].values)
    
    for fold in range(Nfolds):
        
        print("*"*10, f'Fold-{fold+1}', "*"*10,)
      
        X_train = train_df.loc[train_df.fold!=fold, train_cols]
        y_train = train_df.loc[train_df.fold!=fold, TARGET]
        X_val = train_df.loc[train_df.fold==fold, train_cols]
        y_val = train_df.loc[train_df.fold==fold, TARGET]
        
        model = base_model(hidden_units=(256, 512, 1024, 512, 256), embedding_size=16, #16
                           train_cols=train_cols, categorical_cols=cat_cols, 
                          uniques=uniques)
    
        model.compile(
            keras.optimizers.Adam(learning_rate=0.002),
            loss=mean_absolute_error
        )


        train_inputs=[]
        for col in cat_cols:
            train_inputs.append(X_train[col].values)
    
        train_inputs.append(X_train[cont_cols].values)

        val_inputs=[]
        for col in cat_cols:
            val_inputs.append(X_val[col].values)

        val_inputs.append(X_val[cont_cols].values)
        
#         if os.path.isfile(model_path + f'{model_name}_{fold}.h5'):
#             model =  keras.models.load_model(model_path + f'{model_name}_{fold}.h5', 
#                                              custom_objects={'swish': swish, 'Activation': Activation, 
#                                                              'mean_absolute_error':mean_absolute_error})
        
#         else:

        model.fit(train_inputs, 
                  y_train.values,               
                  batch_size=1024,
                  epochs=1000,
                  validation_data=(val_inputs, y_val.values),
                  callbacks=[es, plateau],
                  validation_batch_size=len(y_val),
                  shuffle=True, verbose = 1)

        model.save(f"{model_name}_{fold}.h5")
            
        preds = model.predict(val_inputs).reshape(1,-1)[0]
        
        score = mean_absolute_error(y_val.values, preds)
        score_pvt = mean_absolute_percentage_error(y_val.values, preds)
        scores.append(score)
        scores_pvt.append(score_pvt)
        oof_keras[X_val.index] = preds
        
        print(f'MAE for Fold-{fold+1}:', np.round(score, 3))
        print(f'MAPE for Fold-{fold+1}:', np.round(score_pvt, 3))
        
        pred_keras += model.predict(test_inputs).reshape(1,-1)[0]/Nfolds
        
    score = mean_absolute_error(train_df[TARGET],oof_keras)
    score_pvt = mean_absolute_percentage_error(train_df[TARGET],oof_keras)
    print(f'OOF MAE:', np.round(score, 3))
    print(f'Average MAE:', f'{np.round(np.mean(scores), 3)}+/-{np.round(np.std(scores), 3)}')
    print(f'OOF MAPE:', np.round(score_pvt, 3))
    print(f'Average MAPE:', f'{np.round(np.mean(scores_pvt), 3)}+/-{np.round(np.std(scores_pvt), 3)}')
    
        
    return oof_keras, pred_keras

In [ ]:
set_seed(42)
seed_everything(seed=42)
tf.random.set_seed(42)

if Inference:
    model_path = '../input/ecm-output-of-final-notebook/'
else:
    model_path = './'
    
oof_pred_keras, pred_keras = fit_keras_nn(Nfold, train_df, test_df, train_cols, categorical_cols, TARGET, model_path)
train_df = train_df.copy()
train_df['oof_keras'] = oof_pred_keras

In [ ]:
set_seed(42)
seed_everything(seed=42)
tf.random.set_seed(42)

submission_keras = pd.DataFrame(data = {'Time': test_df['ID'].values, 'Energy': pred_keras})
print(submission_keras.head())
submission_keras.to_csv('submission_keras.csv', index=False)

In [ ]:
# OOF MAE: 0.668
# Average MAE: 0.668+/-0.052
# OOF MAPE: 0.025
# Average MAPE: 0.025+/-0.003

### Writing output

In [ ]:
set_seed(42)
seed_everything(seed=42)
tf.random.set_seed(42)

w1_fastai = 0.5
w1_keras = 0.5
new_pred_ens = train_df['oof_keras']*w1_keras + train_df['oof_fastai']*w1_fastai 
new_pred_ens_hm = (2*train_df['oof_keras']*train_df['oof_fastai'])/(train_df['oof_keras']+train_df['oof_fastai'])
print(f'OOF MAE ENSEMBLE: {mean_absolute_error(train_df.Energy, new_pred_ens)}')
print(f'OOF MAPE ENSEMBLE: {mean_absolute_percentage_error(train_df.Energy, new_pred_ens)}')

print(f'OOF MAE ENSEMBLE HM: {mean_absolute_error(train_df.Energy, new_pred_ens_hm)}')
print(f'OOF MAPE ENSEMBLE HM: {mean_absolute_percentage_error(train_df.Energy, new_pred_ens_hm)}')

new_test_pred_ens = pred_keras*w1_keras + pred_fastai*w1_fastai
new_test_pred_ens_hm = (2*pred_keras*pred_fastai)/(pred_keras+pred_fastai)

submission_ensemble = pd.DataFrame(data = {'Time': test_df['ID'].values, 'Energy': new_test_pred_ens})
print(submission_ensemble.head())
submission_ensemble.to_csv('submission_ensemble.csv', index=False)

submission_ensemble_hm = pd.DataFrame(data = {'Time': test_df['ID'].values, 'Energy': new_test_pred_ens_hm})
print(submission_ensemble_hm.head())
submission_ensemble_hm.to_csv('submission_ensemble_hm.csv', index=False)

